# TP 3 Entrainement d'un agent avec une Q-table

Nous allons créer agent d'apprentissage par renforcement qui apprend à jouer au jeu du pendu en utilisant la méthode Q-learning.

### 1. **Créer la policie : la fonction epsilon_greedy_policy**

Vous devez implémenter une fonction epsilon_greedy_policy qui sélectionne une action selon une stratégie d'exploration-exploitation basée sur la méthode epsilon-greedy. L'agent choisit une action soit en explorant de manière aléatoire, soit en exploitant les connaissances déjà acquises (c'est-à-dire en sélectionnant l'action ayant la meilleure valeur Q pour l'état donné).

**Paramètres d'entrée** :

- state: un tuple contenant deux éléments :
  - state[0]: l'**indice** de l'état courant dans la Q-table.
  - state[1]: une liste ou un tableau indiquant les actions interdites (lettres déjà jouées). Les indices des lettres déjà jouées sont `True`, si la lettre n'a pas encore était jouée ce sera `False`.
- q_table: la table des des valeurs d'état-action Q utilisée pour stocker les valeurs Q pour chaque paire état-action, qui lui permet à l'agent d'estimer les récompenses associées à chaque action dans un état donné.
- epsilon: le paramètre epsilon pour la politique epsilon-greedy, par défaut à 0.3. Il permet d'ajuster la proportion d'actions aléatoires (exploration) par rapport aux actions basées sur les valeurs maximales dans la Q-Table (exploitation).

**Fonctionnement Stratégie epsilon-greedy :**

Avec une probabilité égale à epsilon,  on est dans une phase d'**exploration**, l'agent choisit une action aléatoire parmi les actions disponibles (indices de 0 à 25, 26 lettres.). 

Sinon, nous **exploitons** la Q-table disponible, l'agent sélectionne l'action ayant la valeur Q maximale pour l'état courant dans la Q-Table (exploitation).

  _Gestion des actions interdites_ : Si certaines actions sont interdites (c'est-à-dire déjà essayées), elles doivent être "masquées" (en leur attribuant une valeur très basse, comme -inf), afin de s'assurer que l'agent ne choisit pas ces actions.
  
```python
if np.random.uniform(0, 1) < epsilon:
  return np.random.choice(26)
```
**Retour** 
 La fonction retourne l'index de l'action choisie : 0 pour A, 1 pour B,...


### 2. **Initialiser les paramètres et les variables de stockage**



- **Nombre d'épisodes :**

  ```python
  n_episode = 50000
  ```

  Cela définit le nombre total d'épisodes pour lesquels l'agent va s'entraîner.



- **Historique des récompenses et des actions :**

  ```python
  hist_reward = np.zeros(n_episode)         # Récompenses obtenues par épisode
  hist_actions = np.zeros((26, n_episode))  # Lettres choisies à chaque étape
  ```

  - `hist_reward` stocke la récompense totale obtenue à chaque épisode.
  - `hist_actions` enregistre l'ordre des lettres choisies par l'agent.



- **Valeurs initiales :**

  ```python
  init_value_qtable = 0.3 
  epsilon = 0.3
  ```

  - `init_value_qtable` est la valeur initiale de chaque entrée dans la Q-table.
  - `epsilon` est le paramètre pour la politique ε-greedy, contrôlant l'équilibre entre exploration et exploitation.



### 3. **Initialiser la Q-table**

- **Création de la Q-table :**

  ```python
  q_table = np.full((28**max_length, 26), init_value_qtable, dtype=float)
  ```

  - La Q-table est une matrice qui associe chaque état possible à une valeur pour chaque action possible (ici, les 26 lettres de l'alphabet).
  - `28**max_length` représente le nombre total d'états possibles, en tenant compte des lettres déjà devinées et des positions possibles dans le mot.
  - `max_length` correspond à la longueur maximale des mots utilisés dans le jeu. Pour commencer, je vous recommande des petits mots, par exemple 3. 


### 4 Le script d'apprentissage 

#### 4.1 **Initialiser l'environnement**

- **Création de l'environnement du pendu :**

  ```python
  env = HangedManEnv(max_word_size=max_length)
  ```

  - `HangedManEnv` est une classe qui représente l'environnement du jeu du pendu.
  - `max_word_size` définit la longueur maximale des mots que l'agent devra deviner.

#### 4.2 . **Boucle principale d'apprentissage**

- Une itération par épisode :*


##### 4.2.1 **Réinitialiser l'environnement pour chaque épisode**

- **Réinitialisation :**

  ```python
  obs, info = env.reset()
  terminated = False
  step = 1
  ```

  - `env.reset()` réinitialise l'environnement et renvoie l'observation initiale.
  - `terminated` indique si l'épisode est terminé.
  - `step` est un compteur pour le nombre d'actions dans l'épisode.

##### 4.2.3 **Conversion de l'observation en indice d'état :**

L'état est la version codée ou transformée de l'observation, qui peut être un index ou une représentation plus simple (discrète ou continue) que l'algorithme peut utiliser pour apprendre une politique. dans notre cas, avec une Q-table l'état représente un index.

  ```python
  state_index = np.dot(obs[0], [28**i for i in range(max_length)])
  ```

  - Cette opération convertit l'observation actuelle en un indice unique pour accéder à la Q-table.

Pour aller plus loin : aller voir le notebook : 3-Observation_to_state.ipynb



#### 4.2.4 **Boucle interne jusqu'à la fin de l'épisode**


- Choisir une action selon la politique ε-greedy (epsilon_greedy_policy) définie

- Exécuter l'action choisie dans l'environnement :


- **Mettre à jour la Q-table**

  - **Calculer l'indice du nouvel état :**
  - **Trouver la meilleure action future avec la q_table:**
  - **Calculer la cible de TD (Temporal Difference) :**
        ```python
        td_target = reward + q_table[next_state_index][best_next_action]
        ```
- **Calculer l'erreur de TD :**

   L'erreur de Temparal Différence mesure l'ajustement que l'agent doit faire dans ses estimations des valeurs des états ou des actions en fonction de la récompense qu'il vient de recevoir. Elle permet à l'agent de corriger progressivement ses estimations de valeur à chaque étape, au lieu d'attendre la fin d'un épisode.

  ```python
  td_error = td_target - q_table[state_index][action]
  ```

- **Mettre à jour la Q-table :**

  ```python
  q_table[state_index][action] += q_table[state_index][action] + td_error
  ```

  Cette mise à jour suit l'équation de mise à jour du Q-learning sans facteur d'apprentissage explicite (alpha = 1).

- **Préparer pour le prochain état :**

  ```python
  state_index = next_state_index
  ```

- Enregistrer les données pour l'analyse

- Sauvegarder les actions et les récompenses :

####  **Fin de l'épisode**

- La boucle while se termine lorsque l'agent a deviné le mot ou épuisé ses tentatives.

### **Fermer l'environnement**
